
<main>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
<article class="day-desc"><h2>--- Day 11: Seating System ---</h2><p>Your plane lands with plenty of time to spare. The final leg of your journey is a ferry that goes directly to the tropical island where you can finally start your vacation. As you reach the waiting area to board the ferry, you realize you're so early, nobody else has even arrived yet!</p>
<p>By modeling the process people use to choose (or abandon) their seat in the waiting area, you're pretty sure you can predict the best place to sit. You make a quick map of the seat layout (your puzzle input).</p>
<p>The seat layout fits neatly on a grid. Each position is either floor (<code>.</code>), an empty seat (<code>L</code>), or an occupied seat (<code>#</code>). For example, the initial seat layout might look like this:</p>
<pre><code>L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
</code></pre>
<p>Now, you just need to model the people who will be arriving shortly. Fortunately, people are entirely predictable and always follow a simple set of rules. All decisions are based on the <em>number of occupied seats</em> adjacent to a given seat (one of the eight positions immediately up, down, left, right, or diagonal from the seat). The following rules are applied to every seat simultaneously:</p>
<ul>
<li>If a seat is <em>empty</em> (<code>L</code>) and there are <em>no</em> occupied seats adjacent to it, the seat becomes <em>occupied</em>.</li>
<li>If a seat is <em>occupied</em> (<code>#</code>) and <em>four or more</em> seats adjacent to it are also occupied, the seat becomes <em>empty</em>.</li>
<li>Otherwise, the seat's state does not change.</li>
</ul>
<p><span title="Floor... floor never changes.">Floor (<code>.</code>) never changes</span>; seats don't move, and nobody sits on the floor.</p>
<p>After one round of these rules, every seat in the example layout becomes occupied:</p>
<pre><code>#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##
</code></pre>
<p>After a second round, the seats with four or more occupied adjacent seats become empty again:</p>
<pre><code>#.LL.L#.##
#LLLLLL.L#
L.L.L..L..
#LLL.LL.L#
#.LL.LL.LL
#.LLLL#.##
..L.L.....
#LLLLLLLL#
#.LLLLLL.L
#.#LLLL.##
</code></pre>
<p>This process continues for three more rounds:</p>
<pre><code>#.##.L#.##
#L###LL.L#
L.#.#..#..
#L##.##.L#
#.##.LL.LL
#.###L#.##
..#.#.....
#L######L#
#.LL###L.L
#.#L###.##
</code></pre>
<pre><code>#.#L.L#.##
#LLL#LL.L#
L.L.L..#..
#LLL.##.L#
#.LL.LL.LL
#.LL#L#.##
..L.L.....
#L#LLLL#L#
#.LLLLLL.L
#.#L#L#.##
</code></pre>
<pre><code>#.#L.L#.##
#LLL#LL.L#
L.#.L..#..
#L##.##.L#
#.#L.LL.LL
#.#L#L#.##
..L.L.....
#L#L##L#L#
#.LLLLLL.L
#.#L#L#.##
</code></pre>
<p>At this point, something interesting happens: the chaos stabilizes and further applications of these rules cause no seats to change state! Once people stop moving around, you count <em><code>37</code></em> occupied seats.</p>
<p>Simulate your seating area by applying the seating rules repeatedly until no seats change state. <em>How many seats end up occupied?</em></p>
</article>
<p>Your puzzle answer was <code>2368</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>

In [233]:
with open('input.txt') as input_file:
    seating_arrangement = input_file.read().strip('\n').split('\n')
demo_seating_arrangement = 'L.LL.LL.LL\nLLLLLLL.LL\nL.L.L..L..\nLLLL.LL.LL\nL.LL.LL.LL\nL.LLLLL.LL\n..L.L.....\nLLLLLLLLLL\nL.LLLLLL.L\nL.LLLLL.LL\n'.strip('\n').split('\n')

In [228]:
import numpy as np

def seat_display(seats):
    str = ''
    for row in seats:
        for seat in row:
            str += seat
        str += '\n'
    return str

seats = np.empty((len(seating_arrangement), len(seating_arrangement[0])), dtype=str)
for i,row in enumerate(seating_arrangement):
    for j,seat in enumerate(row):
        seats[i][j] = seat

In [229]:
def change_seats(seats):
    adjacent_seats = np.empty((seats.shape[0], seats.shape[1]), dtype=list)
    changed = False
    
    # first collect statistics about the adjacent seats
    for i,row in enumerate(seats):
        for j,seat in enumerate(row):
            left_bound = max(0, j - 1)
            right_bound = min(len(row), j + 2)
            lower_bound = max(0, i - 1)
            upper_bound = min(len(seats), i + 2)
            adjacent_seats[i][j] = seats[lower_bound:upper_bound, left_bound:right_bound].flatten().tolist()
            
    # then change the seats based on the adjacencies
    for i,row in enumerate(seats):
        for j,seat in enumerate(row):
            taken_seats_count = adjacent_seats[i][j].count('#')
            if seat == '#': taken_seats_count -= 1
            if seat == 'L' and taken_seats_count == 0:
                seats[i][j] = '#'
                changed = True
            elif seat == '#' and taken_seats_count >= 4:
                seats[i][j] = 'L'
                changed = True
                
    return seats, changed

In [230]:
chaos = True
while chaos:
    seats, chaos = change_seats(seats)

In [231]:
def count_occupied_seats(seats):
    unique, counts = np.unique(seats, return_counts=True)
    seat_info = dict(zip(unique, counts))
    return seat_info['#']

In [232]:
occupied_seats = count_occupied_seats(seats)
print('There are %d occupied seats' % occupied_seats)

There are 2368 occupied seats


</p><article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>As soon as people start to arrive, you realize your mistake. People don't just care about adjacent seats - they care about <em>the first seat they can see</em> in each of those eight directions!</p>
<p>Now, instead of considering just the eight immediately adjacent seats, consider the <em>first seat</em> in each of those eight directions. For example, the empty seat below would see <em>eight</em> occupied seats:</p>
<pre><code>.......#.
...#.....
.#.......
.........
..#L....#
....#....
.........
#........
...#.....
</code></pre>
<p>The leftmost empty seat below would only see <em>one</em> empty seat, but cannot see any of the occupied ones:</p>
<pre><code>.............
.L.L.#.#.#.#.
.............
</code></pre>
<p>The empty seat below would see <em>no</em> occupied seats:</p>
<pre><code>.##.##.
#.#.#.#
##...##
...L...
##...##
#.#.#.#
.##.##.
</code></pre>
<p>Also, people seem to be more tolerant than you expected: it now takes <em>five or more</em> visible occupied seats for an occupied seat to become empty (rather than <em>four or more</em> from the previous rules). The other rules still apply: empty seats that see no occupied seats become occupied, seats matching no rule don't change, and floor never changes.</p>
<p>Given the same starting layout as above, these new rules cause the seating area to shift around as follows:</p>
<pre><code>L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
</code></pre>
<pre><code>#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##
</code></pre>
<pre><code>#.LL.LL.L#
#LLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLLL.L
#.LLLLL.L#
</code></pre>
<pre><code>#.L#.##.L#
#L#####.LL
L.#.#..#..
##L#.##.##
#.##.#L.##
#.#####.#L
..#.#.....
LLL####LL#
#.L#####.L
#.L####.L#
</code></pre>
<pre><code>#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##LL.LL.L#
L.LL.LL.L#
#.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLL#.L
#.L#LL#.L#
</code></pre>
<pre><code>#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.#L.L#
#.L####.LL
..#.#.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#
</code></pre>
<pre><code>#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.LL.L#
#.LLLL#.LL
..#.L.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#
</code></pre>
<p>Again, at this point, people stop shifting around and the seating area reaches equilibrium. Once this occurs, you count <em><code>26</code></em> occupied seats.</p>
<p>Given the new visibility method and the rule change for occupied seats becoming empty, once equilibrium is reached, <em>how many seats end up occupied?</em></p>
</article>
<p>Your puzzle answer was <code>2124</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

In [876]:
with open('input.txt') as input_file:
    seating_arrangement = input_file.read().strip('\n').split('\n')

In [877]:
import numpy as np

def seat_display(seats):
    str = ''
    for row in seats:
        for seat in row:
            str += seat
        str += '\n'
    return str

seats = np.empty((len(seating_arrangement), len(seating_arrangement[0])), dtype=str)
for i,row in enumerate(seating_arrangement):
    for j,seat in enumerate(row):
        seats[i][j] = seat

In [878]:
def change_seats(seats):
    h = seats.shape[0]
    w = seats.shape[1]
    seat_views = np.empty((seats.shape[0], seats.shape[1]), dtype=list)
    changed = False
    
    # first collect statistics about the adjacent seats
    for i,row in enumerate(seats):
        for j,seat in enumerate(row):
            
            views = [] # u,r,d,l,ur,dr,dl,ul
            
            # get view above
            u = []
            k = i - 1
            while k >= 0:
                u.append(seats[k][j])
                k -= 1
                
            # get view to right
            r = []
            l = j + 1
            while l < w:
                r.append(seats[i][l])
                l += 1
            
            # get view below
            d = []
            k = i + 1
            while k < h:
                d.append(seats[k][j])
                k += 1
            
            #get view to left
            le = []
            l = j - 1
            while l >= 0:
                le.append(seats[i][l])
                l -= 1
            
            # get diagonal views
            ur = []
            k = i - 1
            l = j + 1
            while k >= 0 and l < w:
                ur.append(seats[k][l])
                k -= 1
                l += 1
            
            dr = []
            k = i + 1
            l = j + 1
            while k < h and l < w:
                dr.append(seats[k][l])
                k += 1
                l += 1
                
            dl = []
            k = i + 1
            l = j - 1
            while k < h and l >= 0:
                dl.append(seats[k][l])
                k += 1
                l -= 1

            ul = []
            k = i - 1
            l = j - 1
            while k >= 0 and l >= 0:
                ul.append(seats[k][l])
                k -= 1
                l -= 1
            
            views += [u] + [r] + [d] + [le] + [ur] + [dr] + [dl] + [ul]
            
            seat_views[i][j] = views
    
    view_counts = np.empty((seats.shape[0], seats.shape[1]), dtype=int)
    
    # get seat counts
    for i,row in enumerate(seats):
        for j,seat in enumerate(row):
            views = seat_views[i][j]
            taken_seats_count = 0
            for view in views:
                view = [seat for seat in view if seat != '.']
                if len(view) != 0 and view[0] == '#':
                    taken_seats_count += 1
                    
            view_counts[i][j] = taken_seats_count
    
    # change seating based on counts
    for i,row in enumerate(seats):
        for j,seat in enumerate(row):

            taken_seats_count = view_counts[i][j]
            views = seat_views[i][j]
            if seat == 'L' and taken_seats_count == 0:
                seats[i][j] = '#'
                changed = True
            elif seat == '#' and taken_seats_count >= 5:
                seats[i][j] = 'L'
                changed = True
                
    return seats, changed

In [879]:
chaos = True
while chaos:
    seats, chaos = change_seats(seats)

In [880]:
occupied_seats = count_occupied_seats(seats)
print('There are %d occupied seats' % occupied_seats)

There are 2124 occupied seats
